In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import numbers
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.formatting.rule import ColorScaleRule
import os

In [2]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [9]:
# ===== Extract the raw data =====
excel_file = "sales_data.xlsx"
processed_df = pd.read_excel(excel_file, sheet_name="Raw Data")

# ===== Transform the data =====
# Step 4.1: Clean data
processed_df["Date"] = pd.to_datetime(processed_df["Date"], errors="coerce").dt.date
processed_df["Quantity"] = pd.to_numeric(processed_df["Quantity"], errors="coerce")
processed_df["Quantity"].fillna(processed_df["Quantity"].mean(), inplace=True)
processed_df["Price"].fillna(processed_df["Price"].mean(), inplace=True)
processed_df["Cost"].fillna(processed_df["Cost"].mean(), inplace=True)
processed_df["Customer"].fillna("Unknown", inplace=True)
processed_df.drop_duplicates(inplace=True)
processed_df["Quantity"] = processed_df["Quantity"].astype(int)

# Step 4.2: Add new columns
processed_df["Revenue"] = processed_df["Quantity"] * processed_df["Price"]
processed_df["Profit"] = processed_df["Revenue"] - (processed_df["Quantity"] * processed_df["Cost"])

# ===== Handle 'Processed_Data' sheet =====
if os.path.exists(excel_file):
    try:
        existing_df = pd.read_excel(excel_file, sheet_name="Processed_Data")
        combined_df = pd.concat([existing_df, processed_df], ignore_index=True)
        combined_df.drop_duplicates(subset=combined_df.columns.tolist(), inplace=True)
        combined_df.sort_values(by="Date", ascending=False, inplace=True) # Sort by Date descending
    except ValueError:
        combined_df = processed_df
else:
    combined_df = processed_df

# ===== Write processed data back =====
with pd.ExcelWriter(excel_file, mode="a" if os.path.exists(excel_file) else "w", engine="openpyxl", if_sheet_exists="replace") as writer:
    combined_df.to_excel(writer, sheet_name="Processed Data", index=False)

# ===== Apply formatting with openpyxl =====
wb = load_workbook(excel_file)
ws = wb["Processed Data"]

# Convert to Excel Table
table = Table(displayName="ProcessedDataTable", ref=ws.dimensions)
style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True, showColumnStripes=False)
table.tableStyleInfo = style
ws.add_table(table)

# Apply currency format to Price, Revenue, Profit
currency_cols = ["Price", "Revenue", "Profit"]
col_index = {cell.value: cell.column for cell in ws[1]}  # map col name -> col index
for col in currency_cols:
    col_letter = ws.cell(row=1, column=col_index[col]).column_letter
    for cell in ws[col_letter]:
        if cell.row > 1:  # skip header
            cell.number_format = numbers.FORMAT_CURRENCY_USD_SIMPLE  # e.g. $1,234

# Apply conditional formatting on Profit (red → green gradient)
profit_col_letter = ws.cell(row=1, column=col_index["Profit"]).column_letter
profit_range = f"{profit_col_letter}2:{profit_col_letter}{ws.max_row}"
rule = ColorScaleRule(
    start_type="min", start_color="FF0000",  # red
    mid_type="percentile", mid_value=50, mid_color="FFFFFF",  # white
    end_type="max", end_color="00B050"  # green
)
ws.conditional_formatting.add(profit_range, rule)

# Autofit columns (approx by setting width to max length)
for col_cells in ws.columns:
    max_length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in col_cells)
    adjusted_width = max_length + 2
    ws.column_dimensions[col_cells[0].column_letter].width = adjusted_width

wb.save(excel_file)

print(f"Processed data saved and formatted in '{excel_file}' (Processed Data).")

C:\Users\aliaa\AppData\Local\Temp\ipykernel_10988\1083339826.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  processed_df["Quantity"].fillna(processed_df["Quantity"].mean(), inplace=True)
C:\Users\aliaa\AppData\Local\Temp\ipykernel_10988\1083339826.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

Processed data saved and formatted in 'sales_data.xlsx' (Processed Data).
